In [2]:
"""Inception-ResNet V1 model for Keras.
# Reference
http://arxiv.org/abs/1602.07261
"""
%tensorflow_version 1.x
import keras
import keras.layers as layers
from keras.datasets import cifar100 

def Conv_Layer(x,filters,filter_size,strides,padding='valid',name=None):
  use_bias=True;
  x = Conv2D(filters,
               filter_size,
               strides=strides,
               padding=padding,
               name=name)(x)
  return x

def stem(input_shape=(32, 32, 3),classes=100):
  #stack layers to build stem architecture
  inputs = Input(shape=input_shape)
  x = Conv_Layer(inputs, 32, 3, strides=2, padding='valid', name='Conv2D_1a_3x3')
  x = Conv_Layer(x, 32, 3, padding='valid', name='Conv2D_2a_3x3')
  x = Conv_Layer(x, 64, 3, name='Conv2D_2b_3x3')
  x = MaxPooling2D(3, strides=2, name='MaxPool_3a_3x3')(x)
  x = Conv_Layer(x, 80, 1, padding='valid', name='Conv2D_3b_1x1')
  x = Conv_Layer(x, 192, 3, padding='valid', name='Conv2D_4a_3x3')
  x = Conv_Layer(x, 256, 3, strides=2, padding='valid', name='Conv2D_4b_3x3')
  return x

def inc_res_block(x, b_type, activation='relu'):
  if b_type == 'B35':
      branch_0 = Conv_Layer(x, 32, 1, name='Conv2d_1x1')
      branch_1 = Conv_Layer(x, 32, 1, name='Conv2d_0a_1x1')
      branch_1 = Conv_Layer(branch_1, 32, 3, name='Conv2d_0b_3x3')
      branch_2 = Conv_Layer(x, 32, 1, name='Conv2d_0a_1x1')
      branch_2 = Conv_Layer(branch_2, 32, 3, name='Conv2d_0b_3x3')
      branch_2 = Conv_Layer(branch_2, 32, 3, name='Conv2d_0c_3x3')
      branches = [branch_0, branch_1, branch_2]
  elif b_type == 'B17':
      branch_0 = Conv_Layer(x, 128, 1, name='Conv2d_1x1')
      branch_1 = Conv_Layer(x, 128, 1, name='Conv2d_0a_1x1')
      branch_1 = Conv_Layer(branch_1, 128, [1, 7], name='Conv2d_0b_1x7')
      branch_1 = Conv_Layer(branch_1, 128, [7, 1], name='Conv2d_0c_7x1')
      branches = [branch_0, branch_1]
  elif b_type == 'B8':
      branch_0 = Conv_Layer(x, 192, 1, name='Conv2d_1x1')
      branch_1 = Conv_Layer(x, 192, 1, name='Conv2d_0a_1x1')
      branch_1 = Conv_Layer(branch_1, 192, [1, 3], name='Conv2d_0b_1x3')
      branch_1 = Conv_Layer(branch_1, 192, [3, 1], name='Conv2d_0c_3x1')
      branches = [branch_0, branch_1]
  else:
      raise ValueError('Unknown Inception-ResNet block type' + str(b_type))

  merged = Concatenate(axis=3)(branches)
  up = Conv_Layer(merged,
                  K.int_shape(x)[3],
                  1,
                  activation=None,
                  use_bias=True)
  up = Lambda(scaling,
              output_shape=K.int_shape(up)[1:],
              arguments={'scale': scale})(up)
  x = add([x, up])
  if activation is not None:
      x = Activation(activation, name='Activation')(x)
  return x

def Inception_ResNetv1(input_shape,classes=100):
  """ function to construct the architecture of Inception_resnetv1 """
  input_x=stem(input_shape=(32, 32, 3),classes=100)

  return model

Using TensorFlow backend.


In [3]:
# Load the CIFAR100 data.
(trainX, trainY), (testX, testY) = cifar100.load_data()

# Input image dimensions.
input_shape = trainX.shape[1:]
print(input_shape)
# Normalize data.
trainX = trainX.astype('float32') / 255
testX = testX.astype('float32') / 255

# # subtract pixel mean
# trainX_mean = np.mean(trainX, axis=0)
# trainX -= trainX_mean
# testX -= trainX_mean

# Convert class vectors to binary class matrices.
trainY = keras.utils.to_categorical(trainY, 100)
testY = keras.utils.to_categorical(testY, 100)

169009152/169001437 [==============================] - 13s 0us/step
(32, 32, 3)


In [0]:
model = Inception_ResNetv1(input_shape=input_shape,classes=100)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()